In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns


In [ ]:
import os

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
filename = "TRAIN.csv"

In [ ]:
homedir = os.path.abspath(os.path.dirname("__name__"))

In [ ]:
fp = os.path.join(homedir, filename)

In [ ]:
df = pd.read_csv(fp)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df['StoreCat'] = df['Store_Type'] + df['Location_Type'] + df['Region_Code']

In [ ]:
# df['Holiday'] = df['Holiday'].astype('category')
# df['Store_id'] = df['Store_id'].astype('category')
# df['Store_Type'] = df['Store_Type'].astype('category')
# df['Location_Type'] = df['Location_Type'].astype('category')
# df['Region_Code'] = df['Region_Code'].astype('category')
# df['Discount'] = df['Discount'].astype('category')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['StoreCat'].nunique()

In [ ]:
storecatdf = df.groupby(['StoreCat']).agg({"Sales":np.mean}).reset_index().rename(columns={"Sales":"StoreCatAvg"})

In [ ]:
storecatdf.columns

In [ ]:
storeiddf = df.groupby(["Store_id"]).agg({"Sales":np.mean}).reset_index().rename(columns={"Sales":"StoreAvg"})

In [ ]:
storeidcatdf = df[['Store_id', 'StoreCat']]

In [ ]:
storeidcatdf.drop_duplicates(inplace=True)

In [ ]:
storeidcatdf = pd.merge(storeidcatdf, storeiddf, on="Store_id", how="left")

In [ ]:
storeidcatdf = pd.merge(storeidcatdf, storecatdf, on="StoreCat", how="left")

In [ ]:
storeidcatdf.head()

In [ ]:
storeidcatdf['Diff'] = storeidcatdf['StoreAvg'] - storeidcatdf['StoreCatAvg']

In [ ]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek

In [ ]:
df['Year'] = df['Year'].astype('category')
df['Month'] = df['Month'].astype('category')

In [ ]:
df['DayOfWeek'].value_counts()

In [ ]:
dayOfWeekDict = {0:"Monday",
                 1:"Tuesday",
                 2:"Wednesday",
                 3:"Thursday",
                 4:"Friday",
                 5:"Saturday",
                 6:"Sunday"}

In [ ]:
df['DayOfWeek'] = df['DayOfWeek'].apply(lambda x : dayOfWeekDict.get(x))

In [ ]:
df2 = df[['Store_id', "StoreCat", "Date", "DayOfWeek", "Holiday", "Discount", "#Order", "Sales"]]

In [ ]:
df2.head()

In [ ]:
for col in df.select_dtypes(include=['category']).columns.tolist():
    print("Number of levels in {} is {}".format(col, df[col].nunique()))

In [ ]:
df['Store_id'].value_counts()

In [ ]:
df['Store_Type'].value_counts()

In [ ]:
df['Location_Type'].value_counts()

In [ ]:
df['Region_Code'].value_counts()

In [ ]:
df['Holiday'].value_counts()

In [ ]:
df['Discount'].value_counts()

In [ ]:
sns.boxplot(df['#Order'])

In [ ]:
sns.boxplot(y='#Order', x="Holiday", hue = 'Discount', data=df)

In [ ]:
sns.boxplot(df['Sales'])

In [ ]:
sns.boxplot(y='Sales', x="Holiday", hue = 'Discount', data=df)

In [ ]:
store1 = df[df['Store_id']==1]

In [ ]:
store1.head()

In [ ]:
for store in df['Store_id'].unique().tolist():
    storedf = df[df['Store_id']==store]
    for col in storedf.select_dtypes(include=['category']).columns.tolist():
        cnt = storedf[col].nunique()
        if (cnt != 1) & (col not in ["Holiday", "Discount", "Year", "Month"]):
            print("Store ID {} : Number of levels in {} is {}".format(store, col, cnt))

In [ ]:
storedf = df2[df2['Store_id']==1]

In [ ]:
storedf.shape

In [ ]:
timeseriesdf = storedf[['Date', 'Sales']]

In [ ]:
timeseriesdf.set_index('Date', inplace=True)

In [ ]:
timeseriesdf.plot()

In [ ]:
ordertsdf = storedf[['Date', '#Order']]

In [ ]:
ordertsdf.set_index('Date', inplace=True)

In [ ]:
ordertsdf.plot()

In [ ]:
corrdf = storedf[['#Order', "Sales"]].corr()

In [ ]:
f = plt.figure(figsize=(19, 15))
plt.matshow(corrdf, fignum=f.number)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
corrdf.style.background_gradient(cmap='coolwarm').set_precision(2)


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression()

In [ ]:
X = storedf[['#Order']]
y = storedf[['Sales']]

In [ ]:
reg.fit(X, y)

In [ ]:
print(reg)

In [ ]:
reg.score(X, y)

In [ ]:
reg.coef_

In [ ]:
reg.intercept_

In [ ]:
from statsmodels.api import OLS

In [ ]:
ols = OLS(y, X)

In [ ]:
ols.fit().summary()

In [ ]:
import statsmodels.api as sm

In [ ]:
#ADF-test(Original-time-series)
res = sm.tsa.adfuller(storedf['Sales'],regression='ct')
print('p-value:{}'.format(res[1]))

In [ ]:
#ADF-test(differenced-time-series)
res = sm.tsa.adfuller(storedf['Sales'].diff().dropna(),regression='c')
print('p-value:{}'.format(res[1]))

In [ ]:
#we use tra.diff()(differenced data), because this time series is unit root process.
fig,ax = plt.subplots(2,1,figsize=(20,10))
fig = sm.graphics.tsa.plot_acf(timeseriesdf.diff().dropna(), lags=60, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(timeseriesdf.diff().dropna(), lags=60, ax=ax[1])
plt.show()

In [ ]:
# resDiff = sm.tsa.arma_order_select_ic(timeseriesdf, max_ar=7, max_ma=7, ic='aic', trend='c')
# print('ARMA(p,q) =',resDiff['aic_min_order'],'is the best.')

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plot_acf(timeseriesdf)
plt.show()

In [ ]:
plot_pacf(timeseriesdf)
plt.show()

In [ ]:
# aggdf = df.groupby(['Store_id', 'Store_Type', 'Location_Type', 
#             'Region_Code', 'Holiday', 'Discount', 'Year', 'Month'])\
#   .agg({"#Order":np.sum, "Sales":np.sum})

In [ ]:
import requests

In [ ]:
from io import BytesIO

In [ ]:
from datetime import datetime

In [ ]:
df2.Date.describe()

In [ ]:
df3 = df2.set_index("Date")

In [ ]:
df3.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le1 = LabelEncoder()
le1.fit(df3.loc[:, 'StoreCat'])
df3['Z'+'StoreCat'] = le1.transform(df3.loc[:, 'StoreCat'])

In [ ]:
le2 = LabelEncoder()
le2.fit(df3.loc[:, 'DayOfWeek'])
df3['Z'+'DayOfWeek'] = le2.transform(df3.loc[:, 'DayOfWeek'])

In [ ]:
le3= LabelEncoder()
le3.fit(df3.loc[:, 'Discount'])
df3['Z'+'Discount'] = le3.transform(df3.loc[:, 'Discount'])

In [ ]:
# colsToEncode = ['StoreCat', 'DayOfWeek', 'Discount']

# for col in colsToEncode:

#     le = LabelEncoder()
#     le.fit(df3.loc[:, col])
#     df3['Z'+col] = le.transform(df3.loc[:, col])

In [ ]:
traindf = df3.loc["2018-01-01":"2019-12-31"]
testdf = df3.loc["2019-01-01":"2019-05-31"]


# traindf =storedf[:trainlen] 
# testdf = storedf[trainlen:] 

In [ ]:
traindf.shape

In [ ]:
testdf.shape

In [ ]:
endog = traindf.loc[:, 'Sales']

In [ ]:
exog = sm.add_constant(traindf.loc[:, ('ZDayOfWeek', 'ZStoreCat', 'ZDiscount', 'Holiday')])

In [ ]:
# Fit the model
mod = sm.tsa.statespace.SARIMAX(endog, exog, order=(1,0,1))
res = mod.fit(disp=False)
print(res.summary())

In [ ]:
texog = sm.add_constant(testdf.loc[:, ('ZDayOfWeek', 'ZStoreCat', 'ZDiscount', 'Holiday')])

In [ ]:
len(texog)

In [ ]:
res.forecast(exog=texog, steps=len(texog))

In [ ]:
pred = res.forecast(exog=texog, steps=len(texog))

In [ ]:
testdf['pred'] = pred.tolist()

In [ ]:
testdf.head()

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
mean_squared_log_error(testdf['Sales'], testdf['pred'])*1000

In [ ]:
validdf = pd.read_csv(os.path.join(homedir, "TEST_FINAL.csv"))

In [ ]:
validdf.shape

In [ ]:
validdf.dtypes

In [ ]:
validdf['StoreCat'] = validdf['Store_Type'] + validdf['Location_Type'] + validdf['Region_Code']

In [ ]:
validdf['Date'] = pd.to_datetime(validdf['Date'])

In [ ]:
validdf['Year'] = validdf['Date'].dt.year
validdf['Month'] = validdf['Date'].dt.month
validdf['DayOfWeek'] = validdf['Date'].dt.dayofweek

In [ ]:
validdf['DayOfWeek'] = validdf['DayOfWeek'].apply(lambda x : dayOfWeekDict.get(x))

In [ ]:
validdf2 = validdf[['Store_id', "StoreCat", "Date", "DayOfWeek", "Holiday", "Discount"]]

In [ ]:
validdf3 = validdf2.set_index("Date")

In [ ]:
validdf3['Z'+'StoreCat'] = le1.transform(validdf3.loc[:, 'StoreCat'])
validdf3['Z'+'DayOfWeek'] = le2.transform(validdf3.loc[:, 'DayOfWeek'])
validdf3['Z'+'Discount'] = le3.transform(validdf3.loc[:, 'Discount'])

In [ ]:
validdf3.head()

In [ ]:
vexog = sm.add_constant(validdf3.loc[:, ('ZDayOfWeek', 'ZStoreCat', 'ZDiscount', 'Holiday')])

In [ ]:
vpred = res.forecast(exog=vexog, steps=len(vexog))

In [ ]:
validdf['Sales'] = vpred.tolist()

In [ ]:
validdf.head()

In [ ]:
datetime.now().microsecond

In [ ]:
validdf[["ID", "Sales"]].to_csv("Submission_{}.csv".format(datetime.now().microsecond), index=None)